In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("Iceberg + S3A (LocalStack)")
    .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.demo.type", "rest")
    .config("spark.sql.catalog.demo.uri", "http://iceberg-rest:8181")
    .config("spark.sql.catalog.demo.warehouse", "s3a://warehouse/wh")
    .config("spark.sql.catalog.demo.io-impl", "org.apache.iceberg.hadoop.HadoopFileIO")  # ✅ not S3FileIO

    # ✅ Force Spark to use correct credentials
    .config("spark.sql.catalog.demo.hadoop.fs.s3a.access.key", "user")
    .config("spark.sql.catalog.demo.hadoop.fs.s3a.secret.key", "password")
    .config("spark.sql.catalog.demo.hadoop.fs.s3a.endpoint", "http://host.docker.internal:4566")
    .config("spark.sql.catalog.demo.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.sql.catalog.demo.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.sql.catalog.demo.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.sql.catalog.demo.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

    .getOrCreate()
)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/23 17:22:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/23 17:22:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df = spark.read.option("header", "true").csv("s3a://warehouse/my_data/logs.csv")
df.show()


25/04/23 17:22:28 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/04/23 17:22:30 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://warehouse/my_data/logs.csv.
java.nio.file.AccessDeniedException: s3a://warehouse/my_data/logs.csv: getFileStatus on s3a://warehouse/my_data/logs.csv: com.amazonaws.services.s3.model.AmazonS3Exception: The AWS Access Key Id you provided does not exist in our records. (Service: Amazon S3; Status Code: 403; Error Code: InvalidAccessKeyId; Request ID: 1K2R0P6WM1TZQ95E; S3 Extended Request ID: aZVn9456cMTS2797Nr0vlHh1SzqcCbZa+FeEIX2YPFEtRGYjQLX9FpU7mxmXmLDCeL1/ImthF9EyN2U8+pjK8jtvG0TUlmqigOAbiUV7Eao=; Proxy: null), S3 Extended Request ID: aZVn9456cMTS2797Nr0vlHh1SzqcCbZa+FeEIX2YPFEtRGYjQLX9FpU7mxmXmLDCeL1/ImthF9EyN2U8+pjK8jtvG0TUlmqigOAbiUV7Eao=:InvalidAccessKeyId
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S

Py4JJavaError: An error occurred while calling o50.csv.
: java.nio.file.AccessDeniedException: s3a://warehouse/my_data/logs.csv: getFileStatus on s3a://warehouse/my_data/logs.csv: com.amazonaws.services.s3.model.AmazonS3Exception: Forbidden (Service: Amazon S3; Status Code: 403; Error Code: 403 Forbidden; Request ID: 1K2HTPMS9QDP57YR; S3 Extended Request ID: hzsaSdjTUXpu9qDmZmhXyaSSXSSQHjwE+6/93/Dlf3vPfehl61AOktNf0eQwW9SR904jPWSt5VVW89DLWQqv5/W8rfagVAZIzOxJE3ST0R8=; Proxy: null), S3 Extended Request ID: hzsaSdjTUXpu9qDmZmhXyaSSXSSQHjwE+6/93/Dlf3vPfehl61AOktNf0eQwW9SR904jPWSt5VVW89DLWQqv5/W8rfagVAZIzOxJE3ST0R8=:403 Forbidden
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:255)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:175)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3796)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3688)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.la